## Prepare

In [ ]:
!pip install open-clip-torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.0 MB/s 
     |████████████████████████████████| 53 kB 1.3 MB/s 
--2022-06-04 09:14:58--  https://thumbs.dreamstime.com/b/lovely-cat-as-domestic-animal-view-pictures-182393057.jpg
Resolving thumbs.dreamstime.com (thumbs.dreamstime.com)... 192.229.163.122
Connecting to thumbs.dreamstime.com (thumbs.dreamstime.com)|192.229.163.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105055 (103K) [image/jpeg]
Saving to: ‘lovely-cat-as-domestic-animal-view-pictures-182393057.jpg’

lovely-cat-as-domes 100%[===================>] 102.59K  --.-KB/s    in 0.02s   

2022-06-04 09:14:59 (4.73 MB/s) - ‘lovely-cat-as-domestic-animal-view-pictures-182393057.jpg’ saved [105055/105055]



In [ ]:
import os
import torch
import torch.nn as nn
from os.path import expanduser  # pylint: disable=import-outside-toplevel
from urllib.request import urlretrieve  # pylint: disable=import-outside-toplevel
def get_aesthetic_model(clip_model="vit_l_14"):
    """load the aethetic model"""
    home = expanduser("~")
    cache_folder = home + "/.cache/emb_reader"
    path_to_model = cache_folder + "/sa_0_4_"+clip_model+"_linear.pth"
    if not os.path.exists(path_to_model):
        os.makedirs(cache_folder, exist_ok=True)
        url_model = (
            "https://github.com/LAION-AI/aesthetic-predictor/blob/main/sa_0_4_"+clip_model+"_linear.pth?raw=true"
        )
        urlretrieve(url_model, path_to_model)
    if clip_model == "vit_l_14":
        m = nn.Linear(768, 1)
    elif clip_model == "vit_b_32":
        m = nn.Linear(512, 1)
    else:
        raise ValueError()
    s = torch.load(path_to_model)
    m.load_state_dict(s)
    m.eval()
    return m

amodel= get_aesthetic_model(clip_model="vit_l_14")
amodel.eval()

import torch
from PIL import Image
import open_clip
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')

## Run

In [ ]:

!wget https://thumbs.dreamstime.com/b/lovely-cat-as-domestic-animal-view-pictures-182393057.jpg

--2022-06-04 09:17:46--  https://thumbs.dreamstime.com/b/lovely-cat-as-domestic-animal-view-pictures-182393057.jpg
Resolving thumbs.dreamstime.com (thumbs.dreamstime.com)... 192.229.163.122
Connecting to thumbs.dreamstime.com (thumbs.dreamstime.com)|192.229.163.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105055 (103K) [image/jpeg]
Saving to: ‘lovely-cat-as-domestic-animal-view-pictures-182393057.jpg.1’

lovely-cat-as-domes 100%[===================>] 102.59K  --.-KB/s    in 0.02s   

2022-06-04 09:17:46 (4.68 MB/s) - ‘lovely-cat-as-domestic-animal-view-pictures-182393057.jpg.1’ saved [105055/105055]



In [ ]:
image = preprocess(Image.open("lovely-cat-as-domestic-animal-view-pictures-182393057.jpg")).unsqueeze(0)
with torch.no_grad():
    image_features = model.encode_image(image)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    prediction = amodel(image_features)
    print(prediction)

tensor([[4.0330]])
